In [1]:
import pandas as pd
import os


In [2]:

# Load orders.csv
orders_df = pd.read_csv('../Staging_1/Orders/orders.csv')
orders_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432 entries, 0 to 1431
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   order_id         1432 non-null   int64 
 1   customer_id      1432 non-null   int64 
 2   order_status     1432 non-null   int64 
 3   order_date       1432 non-null   object
 4   required_date    1432 non-null   object
 5   shipped_date     1432 non-null   object
 6   store_id         1432 non-null   int64 
 7   staff_id         1432 non-null   int64 
 8   source           1432 non-null   object
 9   timestamp_saved  1432 non-null   object
dtypes: int64(5), object(5)
memory usage: 112.0+ KB


In [3]:
# Convert 'required_date' and 'shipped_date' columns to datetime
orders_df['required_date'] = pd.to_datetime(orders_df['required_date'])
orders_df['shipped_date'] = pd.to_datetime(orders_df['shipped_date'])


In [4]:
orders_df['late_delivery'] = orders_df['shipped_date'] > orders_df['required_date']

In [5]:
orders_df

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,source,timestamp_saved,late_delivery
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,DB,2024-05-02 09:21:33.704821,False
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,DB,2024-05-02 09:21:33.704821,False
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,DB,2024-05-02 09:21:33.704821,False
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,DB,2024-05-02 09:21:33.704821,True
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,DB,2024-05-02 09:21:33.704821,False
...,...,...,...,...,...,...,...,...,...,...,...
1427,1474,816,4,2018-03-30,2018-04-01,2018-04-02,1,3,DB,2024-05-02 09:21:33.704821,True
1428,1475,112,4,2018-03-30,2018-04-01,2018-03-31,2,6,DB,2024-05-02 09:21:33.704821,False
1429,1476,35,4,2018-03-31,2018-04-01,2018-04-01,2,7,DB,2024-05-02 09:21:33.704821,False
1430,1477,180,4,2018-03-31,2018-04-03,2018-04-01,2,6,DB,2024-05-02 09:21:33.704821,False


In [6]:

orders_df['latency_days'] = (orders_df['shipped_date'] - orders_df['required_date']).clip(pd.Timedelta(0))


In [7]:
orders_df

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,source,timestamp_saved,late_delivery,latency_days
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,DB,2024-05-02 09:21:33.704821,False,0 days
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,DB,2024-05-02 09:21:33.704821,False,0 days
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,DB,2024-05-02 09:21:33.704821,False,0 days
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,DB,2024-05-02 09:21:33.704821,True,1 days
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,DB,2024-05-02 09:21:33.704821,False,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...
1427,1474,816,4,2018-03-30,2018-04-01,2018-04-02,1,3,DB,2024-05-02 09:21:33.704821,True,1 days
1428,1475,112,4,2018-03-30,2018-04-01,2018-03-31,2,6,DB,2024-05-02 09:21:33.704821,False,0 days
1429,1476,35,4,2018-03-31,2018-04-01,2018-04-01,2,7,DB,2024-05-02 09:21:33.704821,False,0 days
1430,1477,180,4,2018-03-31,2018-04-03,2018-04-01,2,6,DB,2024-05-02 09:21:33.704821,False,0 days


In [8]:
orders_df['weekend'] = orders_df['shipped_date'].dt.dayofweek.isin([6, 0])
orders_df

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,source,timestamp_saved,late_delivery,latency_days,weekend
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,DB,2024-05-02 09:21:33.704821,False,0 days,True
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,DB,2024-05-02 09:21:33.704821,False,0 days,True
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,DB,2024-05-02 09:21:33.704821,False,0 days,True
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,DB,2024-05-02 09:21:33.704821,True,1 days,False
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,DB,2024-05-02 09:21:33.704821,False,0 days,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1427,1474,816,4,2018-03-30,2018-04-01,2018-04-02,1,3,DB,2024-05-02 09:21:33.704821,True,1 days,True
1428,1475,112,4,2018-03-30,2018-04-01,2018-03-31,2,6,DB,2024-05-02 09:21:33.704821,False,0 days,False
1429,1476,35,4,2018-03-31,2018-04-01,2018-04-01,2,7,DB,2024-05-02 09:21:33.704821,False,0 days,True
1430,1477,180,4,2018-03-31,2018-04-03,2018-04-01,2,6,DB,2024-05-02 09:21:33.704821,False,0 days,True


In [9]:
file_path = '../Staging_2/Orders/orders.csv'

os.makedirs(os.path.dirname(file_path), exist_ok=True)
orders_df.to_csv(file_path, index = False) 